<a href="https://colab.research.google.com/github/arofenitra/Scientific-Computing/blob/main/image_processing/image_compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definition, methods
- Goal : To reduce the cost of storage of a digital image.

- Lossy and lossless image compression
Lossless compression when no information is lost during the compression-decompression process, The perfect recovery of the original image is achieved. Otherwise it is lossy compression.
- Methods : Lossy compression, using Transform coding (DCT compression and Wavelet based image compression)  

<img src="https://www.researchgate.net/publication/343751727/figure/fig1/AS:964615429173252@1606993648698/mage-Compression-Method.ppm">  

Wavelet based compression:   
<img src="https://www.researchgate.net/publication/3308674/figure/fig1/AS:654411101638656@1533035173066/a-Block-diagram-of-the-JPEG-2000-encoder-algorithm-b-Dataflow.png">  

DCT based compression:  
<img src="https://www.researchgate.net/publication/229038449/figure/fig4/AS:667829439311887@1536234354786/Block-diagram-of-sequential-JPEG-encoder-and-decoder-JPEG-Encoder-The-original-samples.png">  


### JPEG Image compression
We will follow the algorithm :


- **JPEG Compression**:

1. **Color Space Conversion**:  
The input image is converted from RGB (Red, Green, Blue) color space to YCbCr (Luminance and Chrominance) color space.
Chroma Subsampling: The chrominance components (Cb and Cr) are subsampled by a factor of 2 in both horizontal and vertical directions, resulting in a reduced-resolution image.
2. **Discrete Cosine Transform (DCT)**:  
 The luminance (Y) and chrominance (Cb and Cr) components are divided into 8x8 blocks and transformed using the DCT, which converts the spatial domain data into frequency domain data.
3. **Quantization**:  
The DCT coefficients are quantized to reduce the precision of the coefficients and discard some of the high-frequency components.
4. **Zigzag Reordering**:   
The quantized coefficients are reordered in a zigzag pattern to group the low-frequency coefficients together.
5. **Run-Length Encoding (RLE)**:  
The reordered coefficients are encoded using RLE, which replaces sequences of zeros with a single value.
6. **Huffman Coding**:  
The RLE-encoded coefficients are encoded using Huffman coding, which assigns variable-length codes to the coefficients based on their frequency of occurrence.
6. **Bitstream Formation**:  
The Huffman-coded coefficients are formed into a bitstream, which is the compressed JPEG image.  

- **JPEG Decompression**:

7. **Bitstream Extraction**:  
The compressed JPEG image is extracted from the bitstream.
8. **Huffman Decoding**:  
The Huffman-coded coefficients are decoded using Huffman decoding, which reverses the variable-length coding.
9. **Run-Length Decoding**:  
The RLE-encoded coefficients are decoded using RLE decoding, which replaces the single values with sequences of zeros.
10. **Zigzag Reordering**:  
The reordered coefficients are reordered in the original zigzag pattern.
11. **Inverse Quantization**:  
The quantized coefficients are inverse quantized to restore the original precision.
12. **Inverse Discrete Cosine Transform (IDCT)**:  
The IDCT is applied to the inverse quantized coefficients to transform the frequency domain data back into spatial domain data.
13. **Chroma Upsampling**:  
The chrominance components (Cb and Cr) are upsampled to their original resolution.
14. **Color Space Conversion**:  
The YCbCr image is converted back to RGB color space.
15. **Image Reconstruction**: The decompressed image is reconstructed from the RGB components.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.fftpack import dct, idct
from huffman import HuffmanCoding

# Load an example image
image = Image.open('example.jpg')
image = image.convert('RGB')
image_array = np.array(image)

# Step 1: Color Space Conversion (RGB to YCbCr)
def rgb_to_ycbcr(image):
    r, g, b = image[:, :, 0], image[:, :, 1], image[:, :, 2]
    y = 0.299 * r + 0.587 * g + 0.114 * b
    cb = -0.1687 * r - 0.3313 * g + 0.5 * b + 128
    cr = 0.5 * r - 0.4187 * g - 0.0813 * b + 128
    return np.stack((y, cb, cr), axis=2)

ycbcr_image = rgb_to_ycbcr(image_array)

# Step 2: Chroma Subsampling
def chroma_subsampling(ycbcr_image):
    y, cb, cr = ycbcr_image[:, :, 0], ycbcr_image[:, :, 1], ycbcr_image[:, :, 2]
    cb_sub = cb[::2, ::2]
    cr_sub = cr[::2, ::2]
    return y, cb_sub, cr_sub

y, cb_sub, cr_sub = chroma_subsampling(ycbcr_image)

# Step 3: Discrete Cosine Transform (DCT)
def apply_dct(component):
    h, w = component.shape
    dct_blocks = np.zeros((h // 8, w // 8, 8, 8))
    for i in range(h // 8):
        for j in range(w // 8):
            block = component[i*8:(i+1)*8, j*8:(j+1)*8]
            dct_blocks[i, j] = dct(dct(block.T, norm='ortho').T, norm='ortho')
    return dct_blocks

y_dct = apply_dct(y)
cb_dct = apply_dct(cb_sub)
cr_dct = apply_dct(cr_sub)

# Step 4: Quantization
def quantize(dct_blocks, quantization_matrix):
    return np.round(dct_blocks / quantization_matrix)

quantization_matrix = np.array([
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
])

y_quantized = quantize(y_dct, quantization_matrix)
cb_quantized = quantize(cb_dct, quantization_matrix)
cr_quantized = quantize(cr_dct, quantization_matrix)

# Step 5: Zigzag Reordering
def zigzag(block):
    return block.flatten()[np.argsort([sum([(x//8)*8 + (y%8) for x, y in zip(range(8), range(8))])])]

y_zigzag = np.array([zigzag(block) for block in y_quantized.reshape(-1, 8, 8)])
cb_zigzag = np.array([zigzag(block) for block in cb_quantized.reshape(-1, 8, 8)])
cr_zigzag = np.array([zigzag(block) for block in cr_quantized.reshape(-1, 8, 8)])

# Step 6: Run-Length Encoding (RLE)
def run_length_encode(zigzag_coefficients):
    encoded = []
    count = 0
    for coeff in zigzag_coefficients:
        if coeff == 0:
            count += 1
        else:
            if count > 0:
                encoded.append((0, count))
                count = 0
            encoded.append((coeff, 0))
    if count > 0:
        encoded.append((0, count))
    return encoded

y_rle = run_length_encode(y_zigzag.flatten())
cb_rle = run_length_encode(cb_zigzag.flatten())
cr_rle = run_length_encode(cr_zigzag.flatten())

# Step 7: Huffman Coding
huffman = HuffmanCoding()
y_huffman = huffman.compress(y_rle)
cb_huffman = huffman.compress(cb_rle)
cr_huffman = huffman.compress(cr_rle)

# Step 8: Bitstream Formation
bitstream = y_huffman + cb_huffman + cr_huffman

# JPEG Decompression

# Step 1: Bitstream Extraction
y_huffman_decoded, cb_huffman_decoded, cr_huffman_decoded = bitstream[:len(y_huffman)], bitstream[len(y_huffman):len(y_huffman)+len(cb_huffman)], bitstream[len(y_huffman)+len(cb_huffman):]

# Step 2: Huffman Decoding
y_rle_decoded = huffman.decompress(y_huffman_decoded)
cb_rle_decoded = huffman.decompress(cb_huffman_decoded)
cr_rle_decoded = huffman.decompress(cr_huffman_decoded)

# Step 3: Run-Length Decoding
def run_length_decode(rle_encoded):
    decoded = []
    for value, count in rle_encoded:
        if value == 0:
            decoded.extend([0] * count)
        else:
            decoded.append(value)
    return np.array(decoded)

y_zigzag_decoded = run_length_decode(y_rle_decoded)
cb_zigzag_decoded = run_length_decode(cb_rle_decoded)
cr_zigzag_decoded = run_length_decode(cr_rle_decoded)

# Step 4: Zigzag Reordering
def inverse_zigzag(reordered_coefficients):
    h, w = reordered_coefficients.shape
    inverse_reorder = np.zeros((h // 8, w // 8, 8, 8))
    for i in range(h // 8):
        for j in range(w // 8):
            block = reordered_coefficients[i*8:(i+1)*8, j*8:(j+1)*8]
            inverse_reorder[i, j] = np.sort(block)
    return inverse_reorder

y_inverse_reordered = inverse_zigzag(y_zigzag_decoded.reshape(-1, 8, 8))
cb_inverse_reordered = inverse_zigzag(cb_zigzag_decoded.reshape(-1, 8, 8))
cr_inverse_reordered = inverse_zigzag(cr_zigzag_decoded.reshape(-1, 8, 8))

# Step 5: Inverse Quantization
def inverse_quantize(quantized_dct_blocks, quantization_matrix):
    return quantized_dct_blocks * quantization_matrix

y_inverse_quantized = inverse_quantize(y_inverse_reordered, quantization_matrix)
cb_inverse_quantized = inverse_quantize(cb_inverse_reordered, quantization_matrix)
cr_inverse_quantized = inverse_quantize(cr_inverse_reordered, quantization_matrix)

# Step 6: Inverse DCT
def inverse_dct(dct_blocks):
    h, w, _, _ = dct_blocks.shape
    idct_blocks = np.zeros((h * 8, w * 8))
    for i in range(h):
        for j in range(w):
            block = idct(idct(dct_blocks[i, j].T, norm='ortho').T, norm='ortho')
            idct_blocks[i*8:(i+1)*8, j*8:(j+1)*8] = block
    return idct_blocks

y_idct = inverse_dct(y_inverse_quantized)
cb_idct = inverse_dct(cb_inverse_quantized)
cr_idct = inverse_dct(cr_inverse_quantized)

# Step 7: Chroma Upsampling
cb_upsampled = np.repeat(np.repeat(cb_idct, 2, axis=0), 2, axis=1)
cr_upsampled = np.repeat(np.repeat(cr_idct, 2, axis=0), 2, axis=1)

# Step 8: Color Space Conversion (YCbCr to RGB)
def ycbcr_to_rgb(ycbcr_image):
    y, cb, cr = ycbcr_image[:, :, 0], ycbcr_image[:, :, 1], ycbcr_image[:, :, 2]
    r = y + 1.402 * (cr - 128)
    g = y - 0.3441 * (cb - 128) - 0.7141 * (cr - 128)
    b = y + 1.772 * (cb - 128)
    return np.stack((r, g, b), axis=2)

# Combine the Y, Cb, and Cr components
ycbcr_reconstructed = np.stack((y_idct, cb_upsampled, cr_upsampled), axis=2)

# Convert YCbCr back to RGB
rgb_reconstructed = ycbcr_to_rgb(ycbcr_reconstructed)

# Clip the values to the valid range [0, 255]
rgb_reconstructed = np.clip(rgb_reconstructed, 0, 255).astype(np.uint8)

# Display the original and reconstructed images
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(image_array)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(rgb_reconstructed)
plt.title('Reconstructed Image')
plt.axis('off')

plt.tight_layout()
plt.show()
